In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

from scipy.optimize import leastsq
import plotly.io as pio
pio.renderers.default = 'notebook+vscode'
from sklearn.cluster import KMeans

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed

def dist_line(x0, point_line1, point_line2):
    """
    distance between point x0 and a line that intersects point_line1, and 
    point_line2
    """
    return np.linalg.norm(np.cross(x0-point_line1,x0-point_line2), axis=-1)/ \
              np.linalg.norm(point_line1-point_line2, axis=-1)

def create_cylinder(z, r):
    """
    construct a point cloud of a cylinder with radius r and length len(z)
    """
    phi = np.linspace(0, 2*np.pi, 100)
    xyz = np.zeros((phi.shape[0]*z.shape[0], 3))
    
    for i in range(phi.shape[0]):
        j = i*z.shape[0]
        xyz[j:j+z.shape[0],0] = np.sin(phi[i])*r 
        xyz[j:j+z.shape[0],1] = np.cos(phi[i])*r
        xyz[j:j+z.shape[0],2] = z

    return xyz

def cylinderFitting(xyz,p,th):
    """
    translate a cylinder aligned with z and unknow radius to match point cloud 
    in xyz
    """   
    x = xyz[:,0]
    y = xyz[:,1]

    fitfunc = lambda p, x, y: (p[0] - x)**2 + (p[1] - y)**2 #fit function
    errfunc = lambda p, x, y: fitfunc(p, x, y) - p[2]**2 #error function 

    est_p , success = leastsq(errfunc, p, args=(x, y), maxfev=1000)

    return est_p

def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

plot_using = 'plotly_light'

%load_ext autoreload
%autoreload 2


In [ ]:
# Some useful color arrays
colour_white = np.array([[1, 1, 1]], dtype=np.float64).swapaxes(1, 0)
colour_red = np.array([[1, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_black = np.array([[0, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_blue = np.array([[0, 0, 1]], dtype=np.float64).swapaxes(1, 0)
colour_green = np.array([[0, 1, 0]], dtype=np.float64).swapaxes(1, 0)

## Load geometries

In [ ]:
# Triangle mesh to point cloud (old)
tmesh = o3d.io.read_triangle_mesh('stls/geom_v01.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd_old = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd_old = pcd_old.paint_uniform_color(colour_green)

# Center points at (0, 0)
pcd_old.translate(-pcd_old.get_center())

# Get point cloud points
pcd_points_old = np.asarray(pcd_old.points)

o3d.visualization.draw_geometries([pcd_old])

print("Extent: \n\t", pcd_old.get_max_bound(), "to",  pcd_old.get_min_bound())

In [ ]:
# Triangle mesh to point cloud (best)
tmesh = o3d.io.read_triangle_mesh('stls/decimated_and_cropped.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd = pcd.paint_uniform_color(colour_green)

# Center points at (0, 0)
pcd.translate(-pcd.get_center())

o3d.visualization.draw_geometries([pcd])


In [ ]:
# Get point cloud points, 
pcd_points = np.asarray(pcd.points[::100])

pcd_d = o3d.utility.Vector3dVector(pcd_points)
pcd_d = o3d.geometry.PointCloud(pcd_d)

print("Extent: \n\t", pcd.get_max_bound(), "to",  pcd.get_min_bound())

In [ ]:
if plot_using == 'plotly':
    fig = plot_PCs([pcd_points_old, pcd_points])
    fig.show()
elif plot_using == 'open3D':
    pcd_d = pcd_d.paint_uniform_color(colour_blue)
    pcd_old = pcd_old.paint_uniform_color(colour_green)
    o3d.visualization.draw_geometries([pcd_d, pcd_old])
else:
    fig = plot_PCs([pcd_points_old[::500], pcd_points[::10]])
    fig.show()

## Register into old coordinate system of geom_V1

In [ ]:

pcs_x = pcd_points[::5]
pcs = np.zeros_like(pcs_x)
pcs[:,0] = pcs_x[:,2]
pcs[:,1] = pcs_x[:,1]
pcs[:,2] = -pcs_x[:,0] # mirror
A = affinePC_torch(torch.Tensor(pcs), torch.Tensor(pcd_points_old[::100]))

print(len(A.pc_ct))
print(len(A.pc_us))

# fit volumes based on point clouds
dphi = 1800
dz = 200
start = torch.Tensor([40,0,-150, 0, 0, 50])
#start = torch.Tensor([0,0,0, 0, 0, 0])
bounds = []
for p in start[:3]:
    bounds.append((p-dphi, p+dphi))
for p in start[3:]:
    bounds.append((p-dz, p+dz))
A.fit(start=start, 
      bounds=bounds,
      method='naive',
      max_oper=500,
      lr=0.5)
#A.params = torch.Tensor([900, 450, 450, 0, 0, 0])

In [ ]:
# check registrations
print(A.params)
pcd_R = A.apply_aff()
pcd_iR = A.apply_affinv()
plot_PCs([A.pc_us, A.pc_ct, pcd_R, pcd_iR])

In [ ]:
# print results
print(A.center_ct)
#print(A.params)
#params_fit = A.params.detach()
A.params[0] += 10
print(A.params)

In [ ]:
# Triangle mesh to point cloud (reload and apply transform)
tmesh = o3d.io.read_triangle_mesh('stls/cropped_helmet_geometry_lluis_experiment.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd = pcd.paint_uniform_color(colour_white)

# Center points at (0, 0)
pcd.translate(-pcd.get_center())
pcd_pointsx = np.asarray(pcd.points)
pcd_points = np.zeros_like(pcd_pointsx)
pcd_points[:,0] = pcd_pointsx[:,2]
pcd_points[:,1] = pcd_pointsx[:,1]
pcd_points[:,2] = -pcd_pointsx[:,0]
pcd_points = apply_affine_inverse_torch(torch.Tensor(pcd_points),
                                A.params,
                                A.center_ct)
pcd_points = apply_affine_inverse_torch(pcd_points,
                                torch.Tensor([-6,-6,0,0,0,0]),
                                A.center_ct).numpy()

pcd = o3d.utility.Vector3dVector(pcd_points)
pcd = o3d.geometry.PointCloud(pcd)

print("Extent: \n\t", pcd.get_max_bound(), "to",  pcd.get_min_bound())

In [ ]:
# visualise
if plot_using == 'plotly':
    fig = plot_PCs([pcd_points_old, pcd_points])
    fig.show()
elif plot_using == 'open3D':
    pcd = pcd.paint_uniform_color(colour_blue)
    pcd_old = pcd_old.paint_uniform_color(colour_green)
    o3d.visualization.draw_geometries([pcd, pcd_old])
else:
    fig = plot_PCs([pcd_points_old[::500], pcd_points[::1000]])
    fig.show()

## Get the right set of points that belong to the transducers

In [ ]:
# get inner set of points, just guess center
p_guess = np.zeros(3)
p_guess[0] = -75
p_guess[1] = -45
p_guess[2] = 195
center = np.array([p_guess[0], p_guess[1], 0])
pcd_points_recenter = pcd_points - center 

# create initial cylinder
z = np.arange(pcd_points.min(axis=0)[2], pcd_points.max(axis=0)[2], 1)
cyl = create_cylinder(z, p_guess[2])

# initial set of points selection
d = np.sqrt(np.sum(pcd_points_recenter[:,:2]**2, axis=-1))
select_points = pcd_points_recenter[d < p_guess[2]]

# create open3D objects
cyl_ = o3d.utility.Vector3dVector(cyl)
pcd_cyl = o3d.geometry.PointCloud(cyl_)
pcd_cyl = pcd_cyl.paint_uniform_color(colour_red)
select = o3d.utility.Vector3dVector(select_points)
pcd_select = o3d.geometry.PointCloud(select)
pcd_select = pcd_select.paint_uniform_color(colour_blue)

if plot_using == 'plotly':
    fig = plot_PCs([select_points, cyl])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_select, pcd_cyl])
else:
    fig = plot_PCs([select_points[::100], cyl])
    fig.show()

In [ ]:
# fit cylinder to get precise center and radius
p = np.array([0,0,100])
print(p)
print(" ")

#
z_select = select_points[:,2]
select_points = select_points[(z_select < 170) & (z_select > -170)]
print("Fitting cylinder... ")
est_p = cylinderFitting(select_points,p,0.00001)
print("Done!")
print("Estimated Parameters (x_c, y_c, radius):")
print(est_p)

# create cylinder
refined_center = np.array([est_p[0], est_p[1], 0])
r = est_p[2]
select_points_c = select_points - refined_center

z = np.arange(select_points.min(axis=0)[2], select_points.max(axis=0)[2], 1)
cyl_init = create_cylinder(z, r)

# slightly adjust cylinder
cyl = apply_affine_inverse_torch(torch.Tensor(cyl_init),
                                torch.Tensor([0,0,0,0,0,0]),
                                A.center_ct).numpy()

# create open3D objects
select = o3d.utility.Vector3dVector(select_points_c)
pcd_select = o3d.geometry.PointCloud(select)
pcd_select = pcd_select.paint_uniform_color(colour_blue)
cyl_ = o3d.utility.Vector3dVector(cyl)
pcd_cyl = o3d.geometry.PointCloud(cyl_)
pcd_cyl = pcd_cyl.paint_uniform_color(colour_green)

plot_using = 'plotly-short'
if plot_using == 'plotly':
    fig = plot_PCs([select_points_c[::10], cyl])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_select, pcd_cyl])
else:
    fig = plot_PCs([select_points_c[::100], cyl])
    fig.show()

In [ ]:
# refine selection
d_refined = np.sqrt(np.sum(select_points_c[:,:2]**2, axis=-1))
refined_points = select_points_c[(d_refined < 183.2) & (d_refined > 177.5)]

# check extend
print(f"Extent: \n\t {refined_points.min()} to {refined_points.max()}")

# retain
refined_points = refined_points[(refined_points[:,2] < 195) & 
                                (refined_points[:,2] > -169)]

# create open3D objects
refined = o3d.utility.Vector3dVector(refined_points)
pcd_refined = o3d.geometry.PointCloud(refined)
pcd_refined = pcd_refined.paint_uniform_color(colour_blue)

# get bounding box, to verify alignment of cylindrical axis to z-axis
A = pcd_refined.get_minimal_oriented_bounding_box()
A.color = colour_green

box_points = np.asarray(A.get_box_points())
bottom_points = box_points[box_points[:,2] < 0] # get lower plane
normal_z = np.cross(bottom_points[0,:]-bottom_points[1,:], 
                    bottom_points[0,:]-bottom_points[2,:]) # find normal
normal_z /= np.linalg.norm(normal_z)
angle = 180*np.arccos(normal_z[2])/np.pi
print(f"Normal of bottom plane of bounding box: {normal_z}")
print(f"Angle with z-axis {angle}")

print(len(refined_points))

# visualise
if plot_using == 'plotly':
    fig = plot_PCs([refined_points])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined, A])
else:
    fig = plot_PCs([refined_points[::10]])
    fig.show()

In [ ]:
# clean out craps
x = refined_points[:,0]
y = refined_points[:,1]
z = refined_points[:,2]
crap = refined_points[((x > 140) & (x < 163) & (y < 105) & (y > 80)) | (z > 168)]
refined_x = refined_points[((x < 140) | (x > 163) | (y > 105) | (y < 80)) & (z < 168)]

print(len(crap))
fig = plot_PCs([refined_points[::100], refined_x[::100]])
fig.show()

In [ ]:
# turn over
refined_points = refined_x

In [ ]:
# plot and verify
pcd_refined = o3d.utility.Vector3dVector(refined_points)
pcd_refined = o3d.geometry.PointCloud(pcd_refined)
o3d.visualization.draw_geometries([pcd_refined])

In [ ]:
o3d.visualization.draw_geometries([pcd_refined, pcd_cyl])

## clustering round 1: get seeds

In [ ]:
# perform k-means clustering: this does not work super well, but can be optimised
# for example by passing rough centers of transducers, changing tolerances, n_init,
# I haven't played with it too much yet.
#
# this shows in open3D much more clearly than with plotly
n_clusters = 1700
kmeans = KMeans(n_clusters=n_clusters, 
                verbose=1, 
                init='k-means++',
                
                n_init='auto', 
                algorithm='elkan',
                random_state=567898).fit(refined_points)


In [ ]:
labels = kmeans.labels_
print(f"point cloud has {labels.max() + 1} clusters")
colors = plt.get_cmap("tab20")(labels /(labels.max() + 1))

# update open3D objects
pcd_refined.colors = o3d.utility.Vector3dVector(colors[:, :3])

# separate clusters
cluster_points = []
for i in range(n_clusters):
    cluster_points.append(refined_points[labels == i][::50])

# visualise
if plot_using == 'plotly':
    pass
    #fig = plot_PCs(cluster_points)
    #fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined])
else:
    pass
    #fig = plot_PCs(cluster_points[::10])
    #fig.write_html('vis_clusters-1.html')

In [ ]:
# plot cluster centers
c1s = kmeans.cluster_centers_
fig = plot_PCs([c1s])
fig.show()

### Register nominal geometry to cluster centers

In [ ]:
# open nominal geometry
nominal_geometry_points = pgy2npy('coords/omega2-PointReceivers_in_mm.pgy')
nominal_geometry_points.T[[1,2]] = nominal_geometry_points.T[[2,1]] # swap axes
nominal_geometry_points.T[[1,0]] = nominal_geometry_points.T[[0,1]] # swap axes
print(len(nominal_geometry_points))

nominal_geometry = o3d.utility.Vector3dVector(nominal_geometry_points)
pcd_nominal_geometry = o3d.geometry.PointCloud(nominal_geometry)

if plot_using == 'plotly':
    fig = plot_PCs([c1s, nominal_geometry_points])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined, pcd_nominal_geometry])
else:
    fig = plot_PCs([c1s, nominal_geometry_points])
    fig.show()

In [ ]:
# align two point clouds
# center both geometries
center_c1s = c1s.mean(axis=0)
center_nominal = nominal_geometry_points.mean(axis=0)

clusters_centered = c1s - center_c1s
nominal_centered = nominal_geometry_points - center_nominal

# affine
start = torch.Tensor([0,0,2000,0,0,0]) # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful 1600

print('start_fitting...')
N_PC = affinePC_torch(torch.Tensor(clusters_centered), torch.Tensor(nominal_centered))
dphi = 1800
N_PC.fit(start=start, 
         bounds=[(0-dphi,0+dphi),(0-dphi,0+dphi),(900-dphi, 900+dphi)]+[(None,None)]*3, 
         method='naive',
         max_oper=1000,
         lr=0.1)
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{N_PC.params}")
reg_nominal = N_PC.apply_aff()
params = N_PC.params

# construct open3D objects
reg_nom = o3d.utility.Vector3dVector(reg_nominal)
pcd_reg_nom = o3d.geometry.PointCloud(reg_nom)
pcd_reg_nom = pcd_reg_nom.paint_uniform_color(colour_green)
clusters_ = o3d.utility.Vector3dVector(clusters_centered)
pcd_cluster_c = o3d.geometry.PointCloud(clusters_)
pcd_cluster_c = pcd_cluster_c.paint_uniform_color(colour_blue)

o3d.visualization.draw_geometries([pcd_reg_nom, pcd_cluster_c])


In [ ]:
# construct open3D objects
cluster_centers_vis = [clusters_centered]
for c in reg_nominal:
    cluster_centers_vis.append(c.reshape(1,-1))
fig = plot_PCs(cluster_centers_vis)
fig.write_html('vis_centers-2.html')

In [ ]:
clust_list = [point.reshape(1,-1) + center_c1s for point in reg_nominal]
clust_list.append(refined_points[::10])
fig = plot_PCs(clust_list)
fig.write_html('vis_clust-2.html')


In [ ]:
# remove broke transducers
remove = [839, 841, 1064, 1145, 1171, 1326, 1327, 1354, 1379] #[1156, 1524, 1525, 252, 1055]
reg_nominal_clean = []
clust_list_clean = []
reg_removed = []
for i, clust in enumerate(reg_nominal):

    if i in remove:
        reg_removed.append(clust)
        continue

    reg_nominal_clean.append(clust)

# add two more points for random bits of points
#reg_nominal_clean.append((torch.Tensor([-77.2590, 164.5, -159.7]) - center_c1s).float())
#reg_nominal_clean.append((torch.Tensor([31.5, -174, -162]) - center_c1s).float())

print(reg_nominal_clean[-3])
print(reg_nominal_clean[-1])

In [ ]:
# refit with removed pieces
start = torch.Tensor([0,0,0,0,0,0]) # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful [1800,1800,1400,0,0,0]

print('start_fitting...')
N_PC = affinePC_torch(torch.Tensor(clusters_centered), torch.vstack(reg_nominal_clean))
dphi = 1800
N_PC.fit(start=start, 
         bounds=[(0-dphi,0+dphi),(0-dphi, 0+dphi),(0-dphi, 0+dphi)]+[(None,None)]*3, 
         method='naive',
         max_oper=1000,
         lr=0.001)
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{N_PC.params}")
reg_nominal_clean = N_PC.apply_aff()
params = N_PC.params

# cvisualize fits
reg_nom = o3d.utility.Vector3dVector(reg_nominal_clean)
pcd_reg_nom = o3d.geometry.PointCloud(reg_nom)
pcd_reg_nom = pcd_reg_nom.paint_uniform_color(colour_green)
clusters_ = o3d.utility.Vector3dVector(clusters_centered)
pcd_cluster_c = o3d.geometry.PointCloud(clusters_)
pcd_cluster_c = pcd_cluster_c.paint_uniform_color(colour_blue)

o3d.visualization.draw_geometries([pcd_reg_nom, pcd_cluster_c])

## Clustering round 2: get clusters

In [ ]:
#rerun 1705 minus 9 plus 2?
n_clusters = 1696
reg_nominal_clean_x = reg_nominal_clean.numpy()
#np.random.shuffle(reg_nominal_clean_x)
init_c = reg_nominal_clean_x + center_c1s

# add random fluffs
craps = np.array([[53.0, -172.0, -164.0],
                  [72.0, 164.0, -162.0]])
#init_c = np.vstack([init_c, craps])

kmeans_rerun = KMeans(n_clusters=n_clusters, 
                      verbose=1, 
                      init=init_c,
                      n_init=1, 
                      algorithm='elkan').fit(refined_points)

In [ ]:
labels_rerun = kmeans_rerun.labels_
print(f"point cloud has {labels_rerun.max() + 1} clusters")
colors = plt.get_cmap("tab20")(labels_rerun /(labels_rerun.max() + 1))

# update open3D objects
pcd_refined.colors = o3d.utility.Vector3dVector(colors[:, :3])

# separate clusters
cluster_rerun_points = []
for i in range(labels_rerun.max()+1):
    x = refined_points[labels_rerun == i]
    cluster_rerun_points.append(x[::1])

o3d.visualization.draw_geometries([pcd_refined])


In [ ]:
#fig=plot_PCs(cluster_rerun_points)
#fig.write_html('clusters.html')

In [ ]:
# save clusters
clusters_rerun_points_labelled = cluster_rerun_points[0]
lab = np.ones(len(clusters_rerun_points_labelled))*0
clusters_rerun_points_labelled = np.c_[clusters_rerun_points_labelled, lab]

for i in range(1,len(cluster_rerun_points)):
    temp_np = cluster_rerun_points[i]
    lab = np.ones(len(temp_np))*i
    temp_np = np.c_[temp_np, lab]
    clusters_rerun_points_labelled = np.vstack((clusters_rerun_points_labelled, temp_np))

print(clusters_rerun_points_labelled[-5:])
clusters_rerun_points_labelled.tofile('ExtractedTransducerClustersPCs-jul24.npy')

bucket = {"points" : refined_points,
          "labels"  : labels_rerun}
with open(f"ClusteredPCs-jul24.pk", 'wb') as handle:
    pickle.dump(bucket, handle)

In [ ]:
# load clusters
clusters_rerun_points_labelled = np.fromfile('ExtractedTransducerClustersPCs-jul24.npy').reshape((-1,4))
refined_points = clusters_rerun_points_labelled[:,:3]
labels_rerun = clusters_rerun_points_labelled[:,-1]

## Fitting cylinder

In [ ]:
# create transducer object
r = 5 #4.44
center[0] = 0
center[1] = 0

# cap
x = np.linspace(-5,5,50)
y = np.linspace(-5,5,50)
xy = np.zeros((len(x)*len(y),3))
for i in range(len(x)):
    j = i*len(y)
    xy[j:j+len(y),0] = x[i]
    xy[j:j+len(y),1] = y

cap = xy[np.sum(xy**2, axis=-1) < r**2]

# cyl (perhaps disk only is better?)
z = np.linspace(0, 5, 12)
cyl = create_cylinder(z, r)

ref = np.zeros((len(cyl)+len(cap), 3))
ref[:len(cyl)] = cyl
ref[len(cyl):] = cap

print(f"Number of points in reference cylinder {len(ref)}")

In [ ]:
np.save('cap.npy', cap)
np.save('cyl.npy', cyl)

In [ ]:
# check single pointcloud
test = refined_points[labels_rerun==26]
fig = plot_PCs([test])
fig.show()

In [ ]:
# test fitting cap
test = refined_points[labels_rerun==26] # select a decent one, I have not found a way
                                   # to QC this clustering

# remove excess points?
print(f"Number of points in observed cluster {len(test)}")

center_test = np.mean(test, axis=0)
test -= center_test
start = [0,0,0,0,0,0] # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful

print('start_fitting...')
print(f'center cap: {cap.mean(axis=0)}')
Cap_PC = affinePC(test, cap)
Cap_PC.fit(start=start, bounds=[(-2*np.pi,2*np.pi)]*3+[(None,None)]*3, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cap_PC.params}")
reg_pc = Cap_PC.apply_aff()
params_transd = Cap_PC.params.copy()

# construct open3D objects
reg = o3d.utility.Vector3dVector(reg_pc)
pcd_reg = o3d.geometry.PointCloud(reg)
pcd_reg = pcd_reg.paint_uniform_color(colour_blue)
test_ = o3d.utility.Vector3dVector(test)
pcd_test = o3d.geometry.PointCloud(test_)
pcd_test = pcd_test.paint_uniform_color(colour_green)

fig = plot_PCs([reg_pc, test])
fig.show()

In [ ]:
# fitting cylinder
start = params_transd # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful
dx = 2
dphi = 0.2
bounds_t = [(start[3]-dx, start[3]+dx), 
            (start[4]-dx, start[4]+dx),
            (start[5]-dx, start[5]+dx)]
bounds_r = [(start[0]-dphi, start[0]+dphi), 
            (start[1]-dphi, start[1]+dphi),
            (start[2]-dphi, start[2]+dphi)]

# determine polarity
ref[:len(cyl)] = cyl
ref[len(cyl):] = cap

normal_cap = np.cross(reg_pc[0], reg_pc[1]) # take any 2 cap points
normal_cap /= np.linalg.norm(normal_cap)
trans = params_transd[3:]/np.linalg.norm(params_transd[3:])

if np.dot(normal_cap, trans) < 0:
    print("flip cylinder geom...")
    ref[:,2] = -ref[:,2]

print('start_fitting...')
Cyl_PC = affinePC(test, ref)
Cyl_PC.center_ct = np.array([0,0,0])
Cyl_PC.fit(start=start, bounds=bounds_r+bounds_t, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cyl_PC.params}")
regrefined_pc = Cyl_PC.apply_aff()

# construct open3D objects
reg_ = o3d.utility.Vector3dVector(regrefined_pc)
pcd_regrefined = o3d.geometry.PointCloud(reg_)
pcd_regrefined = pcd_regrefined.paint_uniform_color(colour_blue)

fig = plot_PCs([regrefined_pc, test])
fig.show()

In [ ]:
count = 0
labels_fit = list(set(labels_rerun)) 
errors= []

# slow, use script on server!
for label in labels_fit:

    test = refined_points[labels_rerun==label] # select a decent one, I have not found a way
                                    # to QC this clustering

    try: 
        os.mkdir(f"fittedTransducers/{label}/") # create dir
    except:
        pass

    # remove excess points?
    print(f"Number of points in observed cluster {len(test)}")

    center_test = np.mean(test, axis=0)
    test -= center_test
    start = [np.pi/3,0,np.pi/3,0,0,0] # it will be important to have a rough starting point for 
                        # each! the original geomtery will be very helpful

    print('start_fitting...')
    print(f'center cap: {cap.mean(axis=0)}')
    Cap_PC = affinePC(test, cap)
    Cap_PC.fit(start=start, bounds=[(-2*np.pi,2*np.pi)]*3+[(None,None)]*3, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cap_PC.params}")
    reg_pc = Cap_PC.apply_aff()
    params_transd = Cap_PC.params.copy()

    bucket = {"location" : center_test,
              "Objects"  : Cap_PC}
    with open(f"fittedTransducers/{label}/transCap_{label}.pk", 'wb') as handle:
        pickle.dump(bucket, handle)

    fig = plot_PCs([reg_pc, test])
    fig.write_html(f"fittedTransducers/{label}/transCap_{label}.html")

    # refit with extended ref
    start = params_transd # it will be important to have a rough starting point for 
                        # each! the original geomtery will be very helpful
    dx = 2
    dphi = 0.2
    bounds_t = [(start[3]-dx, start[3]+dx), 
                (start[4]-dx, start[4]+dx),
                (start[5]-dx, start[5]+dx)]
    bounds_r = [(start[0]-dphi, start[0]+dphi), 
                (start[1]-dphi, start[1]+dphi),
                (start[2]-dphi, start[2]+dphi)]

    # determine polarity
    ref[:len(cyl)] = cyl
    ref[len(cyl):] = cap

    normal_cap = np.cross(reg_pc[0], reg_pc[1]) # take any 2 cap points
    normal_cap /= np.linalg.norm(normal_cap)
    trans = params_transd[3:]/np.linalg.norm(params_transd[3:])

    if np.dot(normal_cap, trans) < 0:
        print("flip cylinder geom...")
        ref[:,2] = -ref[:,2]

    print('start_fitting...')
    Cyl_PC = affinePC(test, ref)
    Cyl_PC.center_ct = np.array([0,0,0])
    Cyl_PC.fit(start=start, bounds=bounds_r+bounds_t, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cyl_PC.params}")
    regrefined_pc = Cyl_PC.apply_aff()
    loss = Cyl_PC.loss_l2(Cyl_PC.params, Cyl_PC.pc_ct, Cyl_PC.pc_us, Cyl_PC.center_ct, method='naive')
    if loss/len(Cyl_PC.pc_us) > 0.0004:
        count += 1
        errors.append(label)
        print(f"{i}:  {loss/len(Cyl_PC.pc_us)}")

    bucket = {"location" : center_test,
              "Objects"  : Cyl_PC}
    with open(f"fittedTransducers/{label}/transCyl_{label}.pk", 'wb') as handle:
        pickle.dump(bucket, handle)

    fig = plot_PCs([regrefined_pc, test])
    fig.write_html(f"fittedTransducers/{label}/transCyl_{label}.html")

print(errors)

## Refine fits

In [ ]:
def naive_dist(a, b, p):
    """
    ===========================================================================+
    Find matching points from b to a using a minium distance assignment.

    INPUT
    a                           (np.array)(float) point cloud of reference
                                (NUM_PTS_A, 3)
    b                           (np.array)(float) point cloud of reference
                                (NUM_PTS_B, 3)
    p                           (int) order of the p-norm used to calculate 
                                distances between points in a and b.
                       
    OUTPUT
    assignments                 (np.array)(int) mapping of points B to A
                                (NUM_PTS_A,)
    ===========================================================================+
    """

    length_a = a.shape[0]
    length_b = b.shape[0]

    # calculate the benefit matrix between sets ai and bi
    B = np.repeat(b[:,:,np.newaxis],   length_a, axis=2)
    A = np.repeat(a.T[np.newaxis,:,:], length_b, axis=0)
    cost_matrix = (np.linalg.norm(A-B, ord=p, axis=1)**p)

    # find min dist
    d = np.min(cost_matrix, axis=1)
    
    return d

In [ ]:
# refine fitting: remove pointcloud halos from points
labels_fit = np.array([ 442,  546,  685,  993, 1093, 1094, 1124, 1197, 1222, 1248, 1300,
       1302, 1305, 1326, 1351, 1371, 1405, 1431, 1447, 1473, 1481])
flipped = [685] #546
fucked = [1371]
path = "/Users/danielverschueren/Documents/Sonalis/experiments/StartingGeom-july/"

n_sub = 10
count = 0 
errors = []
n_real = 1703

In [ ]:
# test a fit
for i in [681]:
    path_file = path + f"fittedTransducers-1/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    # plot distances
    temp = A_PC.pc_us
    distance = naive_dist(A_PC.pc_ct_r, temp, 2)
    plt.plot(np.sort(distance))
    plt.grid()
    plt.show()

    # cure
    x = temp[:,0]
    y = temp[:,1]
    z = temp[:,2]
    temp_p = temp[distance < 0.2]
    fig = plot_PCs([A_PC.pc_us, temp_p, A_PC.pc_ct_r])
    fig.show()

    clean_temp = temp[(z > -3) & (x < 1.3)]
    fig = plot_PCs([temp, clean_temp])
    fig.show()

## optional: fixing list of transducers

In [ ]:
# fix 1053: split in 2
for i in [685]:

    path_file = f"fittedTransducers-1/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    # plot distances
    temp = A_PC.pc_us
    distance = naive_dist(A_PC.pc_ct_r, temp, 2)
    plt.plot(np.sort(distance))
    plt.grid()
    plt.xlabel('point #')
    plt.ylabel('L2 distance')
    path_file = f"fittedTransducers-1/{i}/error_curve.png"
    plt.savefig(path_file)
    plt.close()

    # cure
    z = A_PC.pc_us[:,2]
    print(A_PC.params)
    
    B_PC = affinePC(A_PC.pc_us[z < -2], A_PC.pc_ct)
    B_PC.center_ct = A_PC.center_ct
    dx = 20
    dphi = 0.5
    start=A_PC.params
    start[1] = np.pi/2 + np.pi
    #start[3] = 0
    start[5] = 0
    bounds_t = [(start[3]-dx, start[3]+dx), 
                (start[4]-dx, start[4]+dx),
                (start[5]-dx, start[5]+dx)]
    bounds_r = [(start[0]-dphi, start[0]+dphi), 
                (start[1]-dphi, start[1]+dphi),
                (start[2]-dphi, start[2]+dphi)]
    bounds = bounds_r + bounds_t
    B_PC.fit(start=start, bounds=bounds, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{B_PC.params}")
    regrefined_pc = B_PC.apply_aff()
    loss_new = B_PC.loss_l2(B_PC.params, B_PC.pc_ct, B_PC.pc_us, B_PC.center_ct, method='naive')
    print(f"{i} {loss/len(A_PC.pc_us)} vs {loss_new/len(B_PC.pc_us)}")

    # save
    bucket = {"location" : loc,
              "Objects"  : B_PC}
    with open(f"fittedTransducers-1/{i}/transCyl_{i}_refitCutoff.pk", 'wb') as handle:
        pickle.dump(bucket, handle)
    
    fig = plot_PCs([regrefined_pc, A_PC.pc_us, B_PC.pc_us])
    fig.write_html(f"fittedTransducers-1/{1700}/transCyl_{1700}_refitCutoff.html")

In [ ]:
# refit selected transducers (use server)
# angle is off a bit: [ 746 626 ]
for i in labels_fit:

    path_file = f"fittedTransducers/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    # plot distances
    temp = A_PC.pc_us
    distance = naive_dist(A_PC.pc_ct_r, temp, 2)
    plt.plot(np.sort(distance))
    plt.grid()
    plt.xlabel('point #')
    plt.ylabel('L2 distance')
    path_file = f"fittedTransducers/{i}/error_curve.png"
    plt.savefig(path_file)
    plt.close()

    # cure
    B_PC = affinePC(A_PC.pc_us[distance < 0.2], A_PC.pc_ct)
    B_PC.center_ct = A_PC.center_ct
    dx = 20
    dphi = 0.05
    start=A_PC.params
    bounds_t = [(start[3]-dx, start[3]+dx), 
                (start[4]-dx, start[4]+dx),
                (start[5]-dx, start[5]+dx)]
    bounds_r = [(start[0]-dphi, start[0]+dphi), 
                (start[1]-dphi, start[1]+dphi),
                (start[2]-dphi, start[2]+dphi)]
    bounds = bounds_r + bounds_t
    B_PC.fit(start=start, bounds=bounds, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{B_PC.params}")
    regrefined_pc = B_PC.apply_aff()
    loss_new = B_PC.loss_l2(B_PC.params, B_PC.pc_ct, B_PC.pc_us, B_PC.center_ct, method='naive')
    print(f"{i} {loss/len(A_PC.pc_us)} vs {loss_new/len(B_PC.pc_us)}")

    # save
    bucket = {"location" : loc,
            "Objects"  : B_PC}
    with open(f"fittedTransducers/{i}/transCyl_{i}_refitCutoff.pk", 'wb') as handle:
        pickle.dump(bucket, handle)
    
    fig = plot_PCs([regrefined_pc, A_PC.pc_us, B_PC.pc_us])
    fig.write_html(f"fittedTransducers/{i}/transCyl_{i}_refitCutoff.html")

In [ ]:
# compile new losses
threshold = 0.00012
losses = []
loss_ids = []
ids = []
for i in range(n_real):
    try:
        path_file = path + f"fittedTransducers/{i}/transCyl_{i}_refitCutoff.pk"
        with open(path_file, 'rb') as f:
            x = pickle.load(f)
        print(f'{i} refit loaded...')
        ids.append(i)
    except:
        path_file = path + f"fittedTransducers/{i}/transCyl_{i}.pk"
        with open(path_file, 'rb') as f:
            x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    temp = A_PC.pc_ct_r[::n_sub] + loc
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    loss_ids.append(i)
    losses.append(loss/len(A_PC.pc_us))

losses = np.array(losses)
loss_ids = np.array(loss_ids)
poor_fits_id = loss_ids[losses > threshold]
poor_fits_losses = losses[losses > threshold]
print(len(losses[losses > threshold]))
print(poor_fits_id)

plt.plot(np.sort(losses))
plt.grid()
plt.show()

In [ ]:
# relabel save
PC_registr = [[] for i in range(n_real)]
PC_labels = [[] for i in range(n_real)]
for i in range(n_real):
    try:
        path_file = path + f"fittedTransducers/{i}/transCyl_{i}_refitCutoff.pk"
        with open(path_file, 'rb') as f:
            x = pickle.load(f)
        print(f'{i} refit loaded...')
    except:
        path_file = path + f"fittedTransducers/{i}/transCyl_{i}.pk"
        with open(path_file, 'rb') as f:
            x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    temp = A_PC.pc_ct_r[::n_sub] + loc
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    PC_registr[i] = temp.copy()
    PC_labels[i] = i*np.ones_like(temp, dtype=int)

fitted_coords = np.array(PC_registr)

fig = plot_PCs(PC_registr)
fig.write_html(f"fittedTransducers/CombinedTransducers.html")

## Relabel the tranducers and map back to NominalCoords

In [ ]:
# reload nominal geom
nominal_geometry_points = pgy2npy('omega2-PointReceivers_in_mm.pgy')
nominal_geometry_points.T[[1,2]] = nominal_geometry_points.T[[2,1]] # swap axes
nominal_geometry_points.T[[1,0]] = nominal_geometry_points.T[[0,1]] # swap axes

origin = np.mean(nominal_geometry_points, axis=0)
fitted_origin = np.mean(fitted_coords, axis=0)
fitted_coords += origin - fitted_origin

dx = 5
dphi = 1
start = [0.,0.,np.pi-0.35,0,0,0]

#start = [ 0., 0., 2.77345857, 46.13457182, 141.2809497, 157.09586469]

bounds_r = [(start[3]-dx, start[3]+dx), (start[4]-dx, start[4]+dx), (start[5]-dx, start[5]+dx)]
bounds_phi = [(start[0]-dphi,start[0]+dphi),(start[1]-dphi,start[1]+dphi),(start[2]-dphi, start[2]+dphi)]
bounds = bounds_phi + bounds_r

print('start_fitting... fitted points')
M_PC = affinePC(nominal_geometry_points, fitted_coords)
M_PC.params = start
M_PC.fit(start=start, bounds=bounds, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, ty, t_z]: \n{M_PC.params}")
reg_fitted = M_PC.apply_aff()

# move back
nominal_geometry_points.T[[0,1]] = nominal_geometry_points.T[[1,0]] # swap axes
nominal_geometry_points.T[[2,1]] = nominal_geometry_points.T[[1,2]] # swap axes

reg_fitted.T[[0,1]] = reg_fitted.T[[1,0]] # swap axes
reg_fitted.T[[2,1]] = reg_fitted.T[[1,2]] # swap axes

fig = plot_PCs([nominal_geometry_points, reg_fitted])
fig.show()
#fig.write_html("registration_coords.html")

In [ ]:
# relabel
import pandas as pd

L_CBs = np.zeros((1705, 5))
m = 0
n = 0
dn = 1

m_max = 26
n = 0
m = 0
for i, point in enumerate(nominal_geometry_points):

    if i == 1586:
        m_max = 17
        m = 17

    # get label
    if (m % m_max) == 0 and i != 0:
        m = m % m_max
        n += 1
        L_CBs[i,:3] = point
        L_CBs[i,3:] = np.array([m,n])
        m += 1
    else:
        L_CBs[i,:3] = point
        L_CBs[i,3:] = np.array([m,n])
        m += 1

pd.DataFrame(L_CBs).to_csv('CoordsNominal.csv', header=None, index=False)

## save coordinates

In [ ]:
N_CBs = np.zeros((1703, 5))
nom_coords = L_CBs[:,:3]
clean_regfitted = reg_fitted
labels = np.zeros(1703)
m = 0
n = 0
dn = 1

m_max = 26
n = 0
m = 0
for i, point in enumerate(clean_regfitted):

    l2 = np.sum((nom_coords - point)**2, axis=1)
    m = np.argmin(l2)

    if m == 77 or m == 129 or m == 155 or m == 1481:
        print('===')
        print(f"m={m}")
        print(f"i={i}")
    N_CBs[i,:3] = point
    N_CBs[i,3:] = L_CBs[m,3:]
    labels[i] = m

ulabel, counts = np.unique(labels, return_counts=True)
print(ulabel[counts > 1])
print(counts[counts > 1])

for i in range(1705):
    if i not in labels:
        print(i)

N_CBs = np.delete(N_CBs, [1062, 1700, 1701], axis=0)
pd.DataFrame(N_CBs).to_csv('CoordsFitted_3CM.csv', header=None, index=False)

In [ ]:
missing = [252, 1064, 1156, 1524, 1525]
print(L_CBs[missing])